In [126]:
import pandas as pd
import requests

In [127]:
schools_url = "https://aprilascuola.provincia.tn.it/sei//api/istituzioneScolastica/istituzioni/ricerca"
search_school_url =  "https://aprilascuola.provincia.tn.it/sei//api/istituzioneScolastica/istituzioni/0CODE"
#search_school_url = "https://www.istruzione.provincia.tn.it/services/sei/api/v1/institutes/students/0CODE"
schools_trentino = pd.read_json(schools_url)
schools_trentino_covid_url = "https://www.vivoscuola.it/Coronavirus/NEWS/Numero-di-sezioni-classi-messe-in-quarantena-preventiva"
elenco_scuole="https://www.vivoscuola.it/facetsearch/datatable_search/istituto_plesso/viv_istituto|viv_plesso|institutetype|legalstate|dirigente_scolastico|direttore|coordinatore_pedagogico|viv_indirizzo|municipality|phone|fax|email|manageremail|officeemail|website|miurcode/extra_tipoistruzione_name____s|miurcode|legalstate|extra_adulti_name____s|extra_comune_name____s|typelocation/265/CSV?simpleQuery=&query=&extra_tipoistruzione_name____s=&miurcode=&legalstate=&extra_adulti_name____s=&show=10&extra_comune_name____s=&typelocation=&show=10"

In [128]:
def getStatusTipoScuola(valore):
    if (type(valore) == tuple):
        valore = valore[0]
    status = False
    if valore is not None:
        status = True
    return status

In [129]:
def getDataStudentsSchool(idschool):
    idschool = str(idschool)
    rdata = {}
    if (len(idschool) > 4):
        search_school = search_school_url.replace("CODE",idschool)
    else:
        search_school = search_school_url.replace("0CODE",idschool)
    json_data = requests.get(search_school).json()
    totalstudents =  json_data['totaleStudenti']
    totalclasses = json_data['totaleClassi']
    #kindinstruction = json_data['tipoIstituzione']
    primaria =  json_data['primaria'],
    secondaria1grado = json_data['secondaria1grado']
    secondaria2grado = json_data['secondaria2grado']
    formazioneprof = json_data['formazioneProf']
    #infanzia = json_data['infanzia']
    if (totalstudents == None):
        totalstudents = 0
    if (totalclasses == None):
        totalclasses = 0
    rdata['alunni'] = totalstudents
    rdata['classi'] = totalclasses
    #rdata['tipoistituzione'] = kindinstruction
    rdata['primaria'] = primaria
    rdata['secondaria1grado'] = secondaria1grado
    rdata['secondaria2grado'] = secondaria2grado
    rdata['formazioneprof'] = formazioneprof
    #rdata['infanzia'] = infanzia
    return rdata


In [130]:
schools_trentino['extra'] = schools_trentino['codiceProvinciale'].apply(getDataStudentsSchool)

In [131]:
schools_trentino = schools_trentino[~schools_trentino.idPadre.isnull()]
schools_trentino = pd.concat([schools_trentino, schools_trentino['extra'].apply(lambda x: pd.Series(x))], axis=1)
schools_trentino['alunni'] = schools_trentino['extra'].apply(lambda x: x['alunni'])
schools_trentino['classi'] = schools_trentino['extra'].apply(lambda x: x['classi'])
schools_trentino['primaria'] = schools_trentino['extra'].apply(lambda x: getStatusTipoScuola(x['primaria']))
schools_trentino['secondaria1grado'] = schools_trentino['extra'].apply(lambda x: getStatusTipoScuola(x['secondaria1grado']))
schools_trentino['secondaria2grado'] = schools_trentino['extra'].apply(lambda x: getStatusTipoScuola(x['secondaria2grado']))
schools_trentino['formazione_professionale'] = schools_trentino['extra'].apply(lambda x: getStatusTipoScuola(x['formazioneprof']))

In [132]:
del schools_trentino['istituzionePadre']
del schools_trentino['extra']
del schools_trentino['idPadre']
del schools_trentino['idobj']
del schools_trentino['formazioneprof']

In [133]:
schools_trentino[['codiceProvinciale','codiceMiur','denominazioneUfficiale','primaria','secondaria1grado','secondaria2grado','formazione_professionale','indirizzo','email','latitudeY','longitudeX','comune','alunni']].to_csv('alunni_scuole_trentino.csv',index=False)

In [134]:
schools_trentino_covid = pd.read_html(schools_trentino_covid_url,header=0)[0]

In [135]:
schools_trentino_covid.to_csv("alunni_scuole_trentino_quarantena.csv",index=False)

In [136]:
def changeDataRilevamento(v):
    if v.find("2021") == -1:
        if (v.split(" ")[1]) == "dicembre":
            v += " 2021"
        else:
            v += " 2022"
    return v


In [137]:
schools_trentino_covid['DATARILEVAMENTO'] = schools_trentino_covid['DATARILEVAMENTO'].apply(changeDataRilevamento)

In [138]:
schools_trentino_covid.rename(columns={
        'DATARILEVAMENTO':'data_rilevamento',
        'SCUOLAPRIMARIA':'scuola_primaria',
        'SCUOLA SECONDARIADI I GRADO':'scuola_secondaria_primo_grado',
        'SCUOLA SECONDARIADI II GRADO':'scuola_secondaria_secondo_grado',
        'TAGESMUTTER':'tagesmutter',
        "NIDID'INFANZIA":'nido_infanzia',
        "SCUOLAINFANZIA":'scuola_infanzia'
        },inplace=True)

In [139]:
def changeDateString(day):
    day = day.split(" ")
    g = day[0]
    if len(g) <= 2:
        if len(g) == 1:
            g = "0" + g
        if len(day) == 2:
            daytmp = day[1].split("\xa0")
            m = daytmp[0]
            a = daytmp[1]
        else:
            m = day[1]
            a = day[2]
    else:
        g = day[0].split("\xa0")[0]
        m = day[0].split("\xa0")[1]
        a = day[1]
    m = m.lower()
    if m == "gennaio":
        m = "01"
    elif m == "febbraio":
        m = "02"
    elif m == "marzo":
        m = "03"
    elif m == "aprile":
        m = "04"
    elif m == "maggio":
        m = "05"
    elif m == "giugno":
        m = "06"
    elif m == "luglio":
        m = "07"
    elif m == "agosto":
        m = "08"
    elif m == "settembre":
        m = "09"
    elif m == "ottobre":
        m = "10"
    elif m == "novembre":
        m = "11"
    elif m == "dicembre":
        m = "12"
    day = g + "/" + m + "/" + a
    return day

In [140]:
schools_trentino_covid['data_rilevamento'] = schools_trentino_covid['data_rilevamento'].apply(changeDateString)

In [141]:
del schools_trentino_covid['TOTALE']

In [142]:
schools_trentino_covid.to_csv("alunni_scuole_trentino_quarantena_rilevamento.csv",index=False)

In [143]:
totale_studenti_primaria = schools_trentino[schools_trentino['primaria']]['alunni'].sum()

In [144]:
totale_studenti_secondaria_primo_grado = schools_trentino[schools_trentino['secondaria1grado']]['alunni'].sum()

In [145]:
totale_studenti_secondaria_secondo_grado = schools_trentino[schools_trentino['secondaria2grado']]['alunni'].sum() + schools_trentino[schools_trentino['formazione_professionale']]['alunni'].sum()

In [146]:
totale_alunni_per_scuole = pd.DataFrame([[totale_studenti_primaria,totale_studenti_secondaria_primo_grado,totale_studenti_secondaria_secondo_grado]],columns=['totale_alunni_primaria','totale_alunni_secondaria_primo_grado','totale_alunni_secondaria_secondo_grado'])

In [147]:
totale_alunni_per_scuole.to_csv("totale_alunni_per_scuole_provincia_autonoma_trento.csv",index=False)

In [148]:
def percentualeAlunniQuaratena(alunni,totale):
    p = (alunni/totale)*100
    p = round(p,1)
    return p

In [149]:
schools_trentino_covid['percentuale_primaria'] = schools_trentino_covid['scuola_primaria'].apply(lambda x: percentualeAlunniQuaratena(x,totale_studenti_primaria))

In [150]:
schools_trentino_covid['percentuale_secondaria_primo_grado']  = schools_trentino_covid['scuola_secondaria_primo_grado'].apply(lambda x: percentualeAlunniQuaratena(x,totale_studenti_secondaria_primo_grado))

In [151]:
schools_trentino_covid['percentuale_secondaria_grado']  = schools_trentino_covid['scuola_secondaria_secondo_grado'].apply(lambda x: percentualeAlunniQuaratena(x,totale_studenti_secondaria_secondo_grado))

In [152]:
totale_alunni_scuole_primaria_primo_secondo_grado = totale_studenti_primaria + totale_studenti_secondaria_primo_grado + totale_studenti_secondaria_secondo_grado

In [157]:
del schools_trentino_covid['totale_alunni']  

In [159]:
schools_trentino_covid[['scuola_primaria','percentuale_primaria','scuola_secondaria_primo_grado','percentuale_secondaria_primo_grado','scuola_secondaria_secondo_grado','percentuale_secondaria_grado']].to_csv("percentuale_alunni_quarantena_scuole_provincia_autonoma_trento.csv",index=False)